In [1]:
from ta import add_all_ta_features
from ta.utils import dropna
from ta.momentum import RSIIndicator
from datetime import *
from klse_scrapper import *
import talib as ta
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go

In [2]:
from ggplot import *
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks as cf
cf.go_offline(connected=True)
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import plotly.express as px
import plotly.offline as py

In [3]:
from_=datetime.combine(date.today()-timedelta(days=180), time(18)).timestamp()
to_=datetime.combine(date.today(), time(9)).timestamp()
event={'symbol':'COMFORT','counter':'2127','from':int(from_),'to':int(to_),'resolution':'D'}
df=klse_price_data(event)
df['Datetime']=pd.to_datetime(df['Datetime'])
df['Close']=df['Close'].astype(float)
df['ts']=df['Datetime'].apply(lambda x: x.timestamp())
df['RSI'] = ta.RSI(df['Close'])

https://www.klsescreener.com/v2/trading_view/history?symbol=2127&resolution=D&from=1578909600&to=1594429200


Using Linear Regression to find divergence

In [58]:
def model_lin_reg(interval,df):
    df_=pd.DataFrame()
    coef_rsi = []
    coef_price = []
    start_datetime = []
    start_RSI = []
    start_price = []
    future_pricechg = []
    future_price = []
    df_notna=df[df['RSI'].notna()]
    #X=np.reshape(list(range(len(df_notna))),(-1,1))
    X=df_notna['ts'].values.reshape(-1,1)
    for i in range(interval-1,len(X)):
        start_ind = i - (interval-1)
        end_ind = i
        X_in = X[start_ind:end_ind]
        Y_rsi_in = df_notna['RSI'].values.reshape(-1,1)[start_ind:end_ind]
        Y_price_in = df_notna['Close'].values.reshape(-1,1)[start_ind:end_ind]
        reg_rsi = LinearRegression().fit(X_in,Y_rsi_in)
        reg_price = LinearRegression().fit(X_in,Y_price_in)
        coef_rsi.append(reg_rsi.coef_.flatten()[0])
        coef_price.append(reg_price.coef_.flatten()[0])
        start_datetime.append(df_notna['Datetime'].iloc[start_ind])
        start_RSI.append(df_notna['RSI'].iloc[start_ind])
        start_price.append(df_notna['Close'].iloc[start_ind])
        #calculate future price change
        future_ind = end_ind + (interval-1)
        if(future_ind <= len(X)-1):
            future_price_ = df_notna['Close'].iloc[future_ind]
            end_price_ = df_notna['Close'].iloc[end_ind]
            future_pricechg.append((future_price_ - end_price_)/end_price_*100)
            future_price.append(future_price_)
        else:
            future_pricechg.append(0)
            future_price.append(0)
    df_['ts'] = df_notna['ts'][interval-1:]
    df_['end_price_' + str(interval)] = df_notna['Close'][interval-1:]
    df_['end_RSI_' + str(interval)] = df_notna['RSI'][interval-1:]
    df_['end_datetime_' + str(interval)] = df_notna['Datetime'][interval-1:]
    df_['coef_rsi_' + str(interval)] = coef_rsi
    df_['coef_price_' + str(interval)] = coef_price
    df_['start_datetime_' + str(interval)] = start_datetime
    df_['start_RSI_' + str(interval)] = start_RSI
    df_['start_price_' + str(interval)] = start_price
    df_['future_price_change_' + str(interval)] = future_pricechg
    df_['future_price_' + str(interval)] = future_price
    return df_

In [48]:
def model_lin_reg1(interval,df):
    df_=pd.DataFrame()
    coef_rsi = []
    coef_price = []
    start_datetime = []
    end_datetime = []
    start_RSI = []
    end_RSI = []
    start_price = []
    end_price = []
    future_pricechg = []
    ts = []
    df_notna=df[df['RSI'].notna()]
    #X=np.reshape(list(range(len(df_notna))),(-1,1))
    X=df_notna['ts'].values.reshape(-1,1)
    for i in range(interval-1,len(X),interval-1):
        start_ind = i - (interval-1)
        end_ind = i
        X_in = X[start_ind:end_ind]
        Y_rsi_in = df_notna['RSI'].values.reshape(-1,1)[start_ind:end_ind]
        Y_price_in = df_notna['Close'].values.reshape(-1,1)[start_ind:end_ind]
        reg_rsi = LinearRegression().fit(X_in,Y_rsi_in)
        reg_price = LinearRegression().fit(X_in,Y_price_in)
        coef_rsi.append(reg_rsi.coef_.flatten()[0])
        coef_price.append(reg_price.coef_.flatten()[0])
        start_datetime.append(df_notna['Datetime'].iloc[start_ind])
        end_datetime.append(df_notna['Datetime'].iloc[end_ind])
        start_RSI.append(df_notna['RSI'].iloc[start_ind])
        end_RSI.append(df_notna['RSI'].iloc[end_ind])
        start_price.append(df_notna['Close'].iloc[start_ind])
        end_price.append(df_notna['Close'].iloc[end_ind])
        #calculate future price change
        future_ind = end_ind + (interval-1)
        if(future_ind <= len(X)-1):           
            future_pricechg.append((df_notna['Close'].iloc[future_ind]-df_notna['Close'].iloc[end_ind])/df_notna['Close'].iloc[end_ind]*100)
        else:
            future_pricechg.append('0')
    df_['coef_rsi_' + str(interval)] = coef_rsi
    df_['coef_price_' + str(interval)] = coef_price
    df_['start_datetime_' + str(interval)] = start_datetime
    df_['end_datetime_' + str(interval)] = end_datetime
    df_['start_RSI_' + str(interval)] = start_RSI
    df_['end_RSI_' + str(interval)] = end_RSI
    df_['start_price_' + str(interval)] = start_price
    df_['end_price_' + str(interval)] = end_price
    df_['future_price_change_' + str(interval)] = future_pricechg
    return df_

In [65]:
interval=3
df_coef=model_lin_reg(interval,df)
df_coef['divergence']=df_coef['coef_rsi_'+str(interval)] * df_coef['coef_price_'+str(interval)]
df_coef['divergence']=df_coef['coef_price_'+str(interval)] / df_coef['coef_rsi_'+str(interval)] 
df_divergence=df_coef[df_coef['divergence']<0]

In [60]:
df_divergence

,ts,end_price_7,end_RSI_7,end_datetime_7,coef_rsi_7,coef_price_7,start_datetime_7,start_RSI_7,start_price_7,future_price_change_7,future_price_7,divergence
51,1.585094e+09,0.705,42.543466,2020-03-25 08:00:00+08:00,0.000008,-4.122953e-08,2020-03-17 08:00:00+08:00,31.359568,0.69,23.404255,0.87,-0.005081
76,1.588118e+09,1.350,81.294613,2020-04-29 08:00:00+08:00,-0.000003,1.495192e-07,2020-04-21 08:00:00+08:00,83.548664,1.23,68.148148,2.27,-0.043031
77,1.588205e+09,1.410,83.317694,2020-04-30 08:00:00+08:00,-0.000004,1.492079e-07,2020-04-22 08:00:00+08:00,84.764263,1.27,56.737589,2.21,-0.040999
87,1.589846e+09,2.050,69.067275,2020-05-19 08:00:00+08:00,-0.000012,5.745203e-07,2020-05-08 08:00:00+08:00,89.323433,1.75,76.585366,3.62,-0.049568
90,1.590106e+09,2.860,81.693990,2020-05-22 08:00:00+08:00,-0.000022,3.601375e-07,2020-05-14 08:00:00+08:00,90.613778,2.38,3.846154,2.97,-0.016461
91,1.590538e+09,3.080,83.833514,2020-05-27 08:00:00+08:00,-0.000007,9.991241e-07,2020-05-15 08:00:00+08:00,85.497338,2.31,19.155844,3.67,-0.151886
97,1.591229e+09,3.670,66.175588,2020-06-04 08:00:00+08:00,-0.000031,3.444333e-07,2020-05-27 08:00:00+08:00,83.833514,3.08,-29.972752,2.57,-0.011209


In [69]:
from_=datetime.combine(date.today()-timedelta(days=180), time(18)).timestamp()
to_=datetime.combine(date.today(), time(9)).timestamp()
event={'symbol':'INARI','counter':'0166','from':int(from_),'to':int(to_),'resolution':'D'}
df=klse_price_data(event)
df['Datetime']=pd.to_datetime(df['Datetime'])
df['Close']=df['Close'].astype(float)
df['ts']=df['Datetime'].apply(lambda x: x.timestamp())
df['RSI'] = ta.RSI(df['Close'])
interval=7
df_coef=model_lin_reg(interval,df)
df_coef['divergence']=df_coef['coef_rsi_'+str(interval)] * df_coef['coef_price_'+str(interval)]
df_coef['divergence']=df_coef['coef_price_'+str(interval)] / df_coef['coef_rsi_'+str(interval)] 
df_divergence=df_coef[df_coef['divergence']<0]
df_divergence

https://www.klsescreener.com/v2/trading_view/history?symbol=0166&resolution=D&from=1578909600&to=1594429200


,ts,end_price_7,end_RSI_7,end_datetime_7,coef_rsi_7,coef_price_7,start_datetime_7,start_RSI_7,start_price_7,future_price_change_7,future_price_7,divergence
20,1.581379e+09,1.73,48.536325,2020-02-11 08:00:00+08:00,-1.569709e-06,7.507508e-09,2020-02-03 08:00:00+08:00,49.152542,1.72,-4.624277,1.65,-4.782739e-03
38,1.583453e+09,1.41,30.222273,2020-03-06 08:00:00+08:00,2.763595e-06,-7.649817e-08,2020-02-27 08:00:00+08:00,32.847017,1.52,-21.276596,1.11,-2.768067e-02
44,1.584317e+09,1.11,21.598456,2020-03-16 08:00:00+08:00,1.436696e-06,-1.720470e-07,2020-03-06 08:00:00+08:00,30.222273,1.41,0.900901,1.12,-1.197519e-01
50,1.585008e+09,1.12,34.839189,2020-03-24 08:00:00+08:00,1.381216e-05,-8.633634e-08,2020-03-16 08:00:00+08:00,21.598456,1.11,8.035714,1.21,-6.250750e-03
51,1.585094e+09,1.13,35.727638,2020-03-25 08:00:00+08:00,2.083566e-05,-9.438086e-09,2020-03-17 08:00:00+08:00,21.270055,1.10,12.389381,1.27,-4.529775e-04
71,1.587514e+09,1.39,54.248813,2020-04-22 08:00:00+08:00,-1.703417e-06,3.179145e-08,2020-04-14 08:00:00+08:00,55.661283,1.37,2.158273,1.42,-1.866333e-02
80,1.588723e+09,1.38,51.524400,2020-05-06 08:00:00+08:00,-1.457593e-07,1.189858e-08,2020-04-27 08:00:00+08:00,54.617797,1.40,-1.449275,1.36,-8.163172e-02
85,1.589501e+09,1.39,52.112570,2020-05-15 08:00:00+08:00,-6.101870e-07,1.504257e-22,2020-05-05 08:00:00+08:00,53.628933,1.40,0.719424,1.40,-2.465240e-16
93,1.590710e+09,1.49,62.026905,2020-05-29 08:00:00+08:00,-2.186713e-07,8.169935e-09,2020-05-19 08:00:00+08:00,50.938276,1.38,6.711409,1.59,-3.736172e-02
100,1.591747e+09,1.71,73.939642,2020-06-10 08:00:00+08:00,-9.215526e-07,7.233796e-08,2020-06-01 08:00:00+08:00,65.464745,1.53,-4.093567,1.64,-7.849575e-02


In [70]:
#Y_pred=reg.predict(X_in)
from plotly.subplots import make_subplots
#fig = go.Figure()
# Create figure with secondary y-axis
fig1 = make_subplots(specs=[[{"secondary_y": True}]])

fig1.add_trace(
    go.Scatter(x=df['Datetime'], y=df['RSI'], name='RSI'),secondary_y=False
)

for index, row in df_divergence.iterrows():
    fig1.add_trace(go.Scatter(x=[row['start_datetime_'+str(interval)],row['end_datetime_'+str(interval)]], y=[row['start_RSI_'+str(interval)], row['end_RSI_'+str(interval)]],
                         line = dict(color='red', width=2, dash='dash')),secondary_y=False)

#fig1.add_trace(
#    go.Scatter(x=df_divergence['Datetime'],y=df_divergence['RSI'],marker=dict(color="Red",size=8),mode="markers"),secondary_y=False
#)

fig1.add_trace(
    go.Scatter(x=df['Datetime'], y=df['Close'], name='Price', yaxis="y2",line= dict(color='green')),secondary_y=True
)

for index, row in df_divergence.iterrows():
    fig1.add_trace(go.Scatter(x=[row['start_datetime_'+str(interval)],row['end_datetime_'+str(interval)]], y=[row['start_price_'+str(interval)], row['end_price_'+str(interval)]],
                         line = dict(color='yellow', width=2, dash='dash'), yaxis="y2"),secondary_y=True)
fig1.show()

### RSI Divergence Analysis